In [22]:
from rdflib import Graph, Dataset, URIRef
import re

In [23]:
datapath = "/home/jupyter-mvogl/mydata/"

In [24]:
# the URI for our named graph that will contain our data
anno_named_graph_uri = URIRef(f'http://example.org/ns/users/malte/genesis-example-geo2')

In [25]:
# read the JSON-LD file with annotations from Recogito into a Graph
anno_graph = Graph()
anno_graph.parse(f'{datapath}gtclbuppjmot3c(1).jsonld', 
                format='json-ld',
                publicId=anno_named_graph_uri)

<Graph identifier=Na7049bacda2e47a0b3285ff1838b5081 (<class 'rdflib.graph.Graph'>)>

In [26]:
set([x for x in anno_graph.predicates()])

{rdflib.term.URIRef('http://purl.org/dc/terms/creator'),
 rdflib.term.URIRef('http://purl.org/dc/terms/issued'),
 rdflib.term.URIRef('http://purl.org/dc/terms/modified'),
 rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#first'),
 rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#rest'),
 rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'),
 rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#value'),
 rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'),
 rdflib.term.URIRef('http://www.w3.org/ns/activitystreams#generator'),
 rdflib.term.URIRef('http://www.w3.org/ns/oa#end'),
 rdflib.term.URIRef('http://www.w3.org/ns/oa#exact'),
 rdflib.term.URIRef('http://www.w3.org/ns/oa#hasBody'),
 rdflib.term.URIRef('http://www.w3.org/ns/oa#hasPurpose'),
 rdflib.term.URIRef('http://www.w3.org/ns/oa#hasSelector'),
 rdflib.term.URIRef('http://www.w3.org/ns/oa#hasSource'),
 rdflib.term.URIRef('http://www.w3.org/ns/oa#hasTarget'),
 rdf

In [27]:
[x for x in anno_graph.namespaces()]

[('brick', rdflib.term.URIRef('https://brickschema.org/schema/Brick#')),
 ('csvw', rdflib.term.URIRef('http://www.w3.org/ns/csvw#')),
 ('dc', rdflib.term.URIRef('http://purl.org/dc/elements/1.1/')),
 ('dcat', rdflib.term.URIRef('http://www.w3.org/ns/dcat#')),
 ('dcmitype', rdflib.term.URIRef('http://purl.org/dc/dcmitype/')),
 ('dcterms', rdflib.term.URIRef('http://purl.org/dc/terms/')),
 ('dcam', rdflib.term.URIRef('http://purl.org/dc/dcam/')),
 ('doap', rdflib.term.URIRef('http://usefulinc.com/ns/doap#')),
 ('foaf', rdflib.term.URIRef('http://xmlns.com/foaf/0.1/')),
 ('odrl', rdflib.term.URIRef('http://www.w3.org/ns/odrl/2/')),
 ('org', rdflib.term.URIRef('http://www.w3.org/ns/org#')),
 ('owl', rdflib.term.URIRef('http://www.w3.org/2002/07/owl#')),
 ('prof', rdflib.term.URIRef('http://www.w3.org/ns/dx/prof/')),
 ('prov', rdflib.term.URIRef('http://www.w3.org/ns/prov#')),
 ('qb', rdflib.term.URIRef('http://purl.org/linked-data/cube#')),
 ('rdf', rdflib.term.URIRef('http://www.w3.org/19

In [28]:
# write the Graph with annotations to a n-triples file
anno_graph.serialize('recogito-genesis-annotations.nt', format='nt', encoding='utf-8')

<Graph identifier=Na7049bacda2e47a0b3285ff1838b5081 (<class 'rdflib.graph.Graph'>)>

In [29]:
# read the n-triples file and replace all blank nodes (starting with _:) with real URIs
# and write that to a new file
with open('recogito-genesis-annotations.nt', 'r') as infile, open('recogito-genesis-annotations-nb.nt', 'w') as outfile:
    for line in infile:
        line = re.sub(r'_:([^ ]+)', r'<http://example.org/ns/users/malte/genesis-example-geo2/bnode_\1>', line)
        print(line, file=outfile)

In [30]:
# open a connection to the Blazegraph triple store
blazegraph_uri = 'http://localhost:8080/blazegraph/namespace/kb/sparql'
store = Dataset('SPARQLUpdateStore')
store.open((blazegraph_uri, blazegraph_uri))

In [31]:
# find our named graph if it exists or create a new one
anno_named_graph = None
for g in store.graphs():
    if g.identifier == anno_named_graph_uri:
        anno_named_graph = g
if anno_named_graph is None:
    anno_named_graph = store.graph(URIRef(anno_named_graph_uri))

In [32]:
# parse our modified n-triples file into our named graph (takes a while)
anno_named_graph.parse('recogito-genesis-annotations-nb.nt')

<Graph identifier=http://example.org/ns/users/malte/genesis-example-geo2 (<class 'rdflib.graph.Graph'>)>

In [33]:
store.close()